# Homework Set 7

**Author**: Chase Coleman

**NYU ID**: N10827183

**Date**: 31 March 2016

In [1]:
import numpy as np

## Preliminary Description

As in the lecture, let dividends be given by $D_t = d(X_t)$ for some state process $\{ X_t \}$. The state process takes values in a countable set $S$, called the state space. We let the transition probabilities for the state be given by

$$q(x, y) := \mathbb{P}\{ X_{t+1} = y | X_t = x\} \quad (x, y \in S)$$

As discussed in the lecture, an equilibrium price function is a function $p$ on $S$ that satisfies

$$p(x) = \beta \sum_{y \in S} [d(y) + p(y)] q(x, y)$$

for all $x \in S$. Assume in all of what follows that $\beta \in (0, 1)$ and that $d$ is any function from $S$ to $(0, \infty)$.

Below I state a theorem on existence and uniqueness that is similar to the classic work of Lucas (1978) on asset pricing, although I'll stick to the risk neutral case to simplify the discussion.

Incidentally, this theorem is not the best we can do, but it's proof is relatively easy. Please read carefully, so you can apply similar ideas in the exercise.

**Theorem** _If $d \in b \mathbb{R}^S$, then there exists a unique equilibrium price function $p^* \in b \mathbb{R}^S$. Moreover, if $p$ is any function in $b \mathbb{R}^S$, then $d_\infty(T^k p, p^*) \rightarrow 0$ as $k \rightarrow \infty$, where_

$$T p(x) = \beta \sum_{y \in S} [d(y) + p(y)] q(x, y) \quad (x \in S)$$

_Proof_: It suffices to show that $T$ is a uniform contraction on $(b \mathbb{R}^S, d_{\infty})$ whenever $d \in b \mathbb{R}^S$. Assuming that $d \in b \mathbb{R}^S$, to show that $T$ has the stated properties, we must first show that $T$ does in fact map $b \mathbb{R}^S$ into itself. To see this, observe that, for any given $p \in b \mathbb{R}^S$ and any $x \in S$,

\begin{align*}
  | T p(x) | &= \beta \sum_{y \in S} [d(y) + p(y)] q(x,y) \\
  &\leq \beta \sum_{y \in S} |d(y) + p(y)| q(x, y) \\
  &\leq \beta \sum_{y \in S}( ||d||_{\infty} + ||p||_{\infty}) q(x, y) \\
  &= \beta ( ||d||_{\infty} + ||p||_{\infty})
\end{align*}

This bound is uniform over $x$ so taking the supremum on the left hand side gives $||Tp||_{\infty} < \infty$. In other words, $Tp \in b \mathbb{R}^S$.

To see that $T$ is a uniform contraction, pick any $p, p' \in b \mathbb{R}^S$ and let $x$ be any element of $S$. We have

\begin{align*}
  | T p(x) - Tp'(x)| &= \beta \sum_{y \in S} [p(x) - p'(x)] q(x, y) \\
  &\leq \beta \sum_{y \in S} ||p - p'||_{\infty} q(x, y) \\
  &= \beta || p - p' ||_{\infty}
\end{align*}

Since $x \in S$ was arbitrary, we have

$$||Tp - Tp'||_{\infty} \leq \beta ||p - p'||_{\infty}$$

Hence $T$ is a uniform contraction on $(b \mathbb{R}^S, d_{\infty})$ as claimed.

## Exercise 1


### Question

Provide a Python or Julia function that takes a specification of the primitives and returns an approximation to the equilibrium price function. You can assume that $S$ is finite. The computation method should be iterative, based around the Banach contraction mapping theorem. That is, choose some initial $p \in b \mathbb{R}^S$ and repeatedly apply $T$ to obtain $T^k p$ for $k=1,2,\dots$. As a stopping rule for the iteration, stop when successive iterates are close in the sense of $d_{\infty}$.

To test your code, suppose that the state process for the economy has three states: normal growth, mild recession, and severe recession. The transition probabilities $q_1(x, y)$ are given by

$$\begin{bmatrix} & NG & MR & SR \\
                  NG & 0.971 & 0.029 & 0.000 \\
                  MR & 0.145 & 0.778 & 0.077 \\
                  SR & 0.000 & 0.508 & 0.492  \end{bmatrix}$$

Set dividends to 1 in state NG and zero otherwise. Set $\beta = 0.98$. Report the corresponding price function.

Now repeat, with new transition probabilities $q_2(x, y)$ given by

$$\begin{bmatrix} & NG & MR & SR \\
                  NG & 0.871 & 0.029 & 0.100 \\
                  MR & 0.145 & 0.778 & 0.077 \\
                  SR & 0.000 & 0.508 & 0.492  \end{bmatrix}$$

How do your answers change? Can you give any interpretation?

### Solution

Below we write code which applies the $T$ operator to solve the pricing problem

In [2]:
def T_operator(beta, d, p, q):
    """
    Applies the T operator once for a given state x
    
    Parameters
    ----------
    beta : scalar(Float64)
        Agent's discount factor
    d : Array(Float64, ndim=1)
        The dividend paid in each state
    p : Array(Float64, ndim=1)
        The current price function
    q : Array(Float64, ndim=1)
        Probability of transition from the state x to
        each possible state y
    
    Returns
    -------
    Tp : scalar(Float64)
        The value of T-operator for a specific x
    """
    Tp = beta * np.dot(d+p, q)
    return Tp

def compute_price_function(beta, d, Q, tol=1e-7, maxit=1000):
    """
    Applies the T operator once for a given state x
    
    Parameters
    ----------
    beta : scalar(Float64)
        Agent's discount factor
    d : Array(Float64, ndim=1)
        The dividend paid in each state
    p : Array(Float64, ndim=1)
        The current price function
    q : Array(Float64, ndim=1)
        Probability of transition from the state x to
        each possible state y
    
    Returns
    -------
    Tp : scalar(Float64)
        The value of T-operator for a specific x
    """
    # Set-up iteration parameters
    niters = 0
    dist = 10.0
    
    # Allocate space to update pricing function
    nstates = d.size
    p = np.ones(nstates)
    Tkp = np.copy(p)
    
    # Repeatedly apply the T operator to get convergence
    # through the Banach contraction theorem
    while (niters < maxit) and (dist > tol):
        
        # Update each state
        niters = niters + 1
        for i in range(nstates):
            # Transition from curr state to tomorrow
            q = Q[i, :]
            Tkp[i] = T_operator(beta, d, p, q)
        
        # Compute distance and update prices
        dist = np.max(np.abs(p - Tkp))
        p[:] = Tkp[:]
        
    return Tkp
        

Define the transition matrices and dividend function as we are instructed in the exercise

In [3]:
Q1 = np.array([[0.971, 0.029, 0.000],
               [0.145, 0.778, 0.077],
               [0.000, 0.508, 0.492]])

Q2 = np.array([[0.871, 0.029, 0.100],
               [0.145, 0.778, 0.077],
               [0.000, 0.508, 0.492]])

d = np.array([1.0, 0.0, 0.0])

beta = 0.98

In [4]:
p1 = compute_price_function(beta, d, Q1)
p1

array([ 40.75976974,  35.96087782,  34.57199775])

In [5]:
p2 = compute_price_function(beta, d, Q2)
p2

array([ 23.50112992,  21.0988263 ,  20.28394791])

Notice that the price in under the first transition matrix is higher than the price under the second transition matrix. Given by

\begin{align*}
  p_1(x) = [ 40.75976974,  35.96087782,  34.57199775] \\
  p_2(x) = [ 23.50112992,  21.0988263 ,  20.28394791] 
\end{align*}

This is related to the persistence of the state which pays dividends -- Recall that you only receive dividends in the normal growth state, so if you are less likely to stay in that state then the asset is worth less to you.

## Harrison Kreps Model

Now consider the asset pricing problem with heterogeneous beliefs described in Ljunqvist and Sargent. The ideas are originally due to Harrison and Kreps (1978). Here we treat only the mathematical and computational aspects of the problem. See the discussion in either of these references for motivation.

The equilibrium condition is modified to

$$p(x) = \beta \max_{i \in \{1,2\}} \sum_{y \in S} [d(y) + p(y)] q_i(x, y) \quad x \in S$$

where $q_1$ and $q_2$ represent different beliefs about state transition probabilities.

## Exercise 2

### Question

Give conditions under which (3) has a unique solution in $b \mathbb{R}^S$ by extending our earlier analysis (i.e., the case of homoegenous beliefs). Do not assume that $S$ is finite. Give a proof that your conditions are sufficient.

Remark: One possibly helpful hint is the following inequality, which holds for any sequences $\{a_i\}$ and $\{b_i\}$.

$$| \max_i a_i - \max_i b_i | \leq \max_i | a_i - b_i |$$

If you use this inequality you should prove that it holds.

### Answer

Define our "Harrison-Kreps" operator by

$$H p(x) = \beta \max_{i \in \{1, 2\} } \left[ \sum_{y \in S} [d(y) + p(y)] q(x, y) \right]$$

Given this operator and the right assumptions, we can prove there exists a unique equilibrium function $p^*$.

**Theorem** _If $d \in b \mathbb{R}^S$, then there exists a unique equilibrium price function $p^* \in b \mathbb{R}^S$ such that for the "Harrison-Kreps operator $H p^* = p^*$. Moreover, if $p$ is any function in $b \mathbb{R}^S$, then $d_\infty(H^k p, p^*) \rightarrow 0$ as $k \rightarrow \infty$,_

_Proof_:

We know that $(b \mathbb{R}^S, d_{\infty})$ is complete. Then it only remains to show that $H$ is a uniform contraction. If $H$ is a uniform contraction then $H$ has a unique fixed point, $p^* \in b \mathbb{R}^S$ and $d_{\infty}(H^k p, p^*) \leq \beta^k d_{\infty}(p, p^*)$.

We first verify that $H$ is a self-map. Let $p \in b \mathbb{R}^S$, then:

\begin{align*}
  H p(x) &\leq | H p(x) | \\
  &= \beta \max_{i} \left| \left[ \sum_{y \in S} [d(y) + p(y)] q_i(x, y) \right] \right| \\
  &\leq \beta \max_{i} \left[ \sum_{y \in S} |d(y) + p(y)| q_i(x, y) \right] \\
  &\leq \beta \max_{i} \left[ \sum_{y \in S} [ ||d(y)||_{\infty} + ||p(y)||_{\infty} ] q_i(x, y) \right] \\
  &= \beta \max_i [ ||d(y)||_{\infty} + ||p(y)||_{\infty} ] \\
  &= \beta [ ||d(y)||_{\infty} + ||p(y)||_{\infty} ]
\end{align*}

Since $d \in b \mathbb{R}^S$ and $p \in b \mathbb{R}^S$ then $\beta [ ||d(y)||_{\infty} + ||p(y)||_{\infty} ] \in b \mathbb{R}^S$ ($H p(x)$ is less than or equal to some bounded number for any $x \in S$).

Next we need to show that $H$ is a uniform contraction. Let $p,p' \in b \mathbb{R}^S$ and let $x \in S$. Note

\begin{align*}
  |H p(x) - H p'(x)| &= \left| \beta \max_{i} \left[ \sum_{y \in S} [d(y) + p(y)] q_i(x, y) \right] - \beta \max_{i} \left[ \sum_{y \in S} [d(y) + p'(y)] q_i(x, y) \right] \right| \\
  &\leq \beta max_i \left| \sum_{y \in S} \left[ [d(y) + p(y)] - [d(y) + p'(y)] \right] q_i(x, y) \right| \\
  &= \beta \max_i \left| \sum_{y \in S} \left[ p(y) - p'(y) \right] q(x, y) \right| \\
  &\leq \beta ||p(y) - p'(y)||_{\infty} \\
\end{align*}

Taking the supremum over $x \in S$ gives us

$$||H p(x) - H p'(x) ||_{\infty} \leq \beta ||p(y) - p'(y)||_{\infty}$$

We assumed $\beta \in (0, 1)$ which means this is a uniform contraction. Our desired results are then obtained using the Banach Fixed Point Theorem.

**Lemma**: Given two sequences, $a_i$ and $b_i$, show that
$$|\max_i a_i - \max_i b_i| \leq \max_i |a_i - b_i|$$

_Proof_: Left unproven

## Exercise 3

Write a second function that does the same job as the function you wrote for exercise 1, but now for the heterogenous beliefs case. For the computation problem you can assume that $S$ is finite, similar to exercise 1. To test your code, let $q_1$ and $q_2$ be the transition probabilities given in that exercise, and let $d$ and $\beta$ be as specified in that exercise. Compute the equilibrium price function.

In [6]:
def hk_T_operator(beta, d, p, q1, q2):
    """
    Applies the T operator once for a given state x
    
    Parameters
    ----------
    beta : scalar(Float64)
        Agent's discount factor
    d : Array(Float64, ndim=1)
        The dividend paid in each state
    p : Array(Float64, ndim=1)
        The current price function
    q1 : Array(Float64, ndim=1)
        Probability of transition from the state x to
        each possible state y in beliefs of agent 1
    q2 : Array(Float64, ndim=1)
        Probability of transition from the state x to
        each possible state y in beliefs of agent 2

    Returns
    -------
    Tp : scalar(Float64)
        The value of T-operator for a specific x
    """
    Tp = max(beta*np.dot(d+p, q1), beta*np.dot(d+p, q2))
    return Tp

def hk_compute_price_function(beta, d, Q1, Q2, tol=1e-7, maxit=1000):
    """
    Applies the T operator once for a given state x
    
    Parameters
    ----------
    beta : scalar(Float64)
        Agent's discount factor
    d : Array(Float64, ndim=1)
        The dividend paid in each state
    Q1 : Array(Float64, ndim=2)
        Probability of transition from the state x to
        each possible state y in agent 1's beliefs
    Q2 : Array(Float64, ndim=2)
        Probability of transition from the state x to
        each possible state y in agent 1's beliefs

    Returns
    -------
    Tp : scalar(Float64)
        The value of T-operator for a specific x
    """
    # Set-up iteration parameters
    niters = 0
    dist = 10.0
    
    # Allocate space to update pricing function
    nstates = d.size
    p = np.ones(nstates)
    Tkp = np.copy(p)
    
    # Repeatedly apply the T operator to get convergence
    # through the Banach contraction theorem
    while (niters < maxit) and (dist > tol):
        
        # Update each state
        niters = niters+1
        for i in range(nstates):
            # Transition from curr state to tomorrow
            q1 = Q1[i, :]
            q2 = Q2[i, :]
            
            Tkp[i] = hk_T_operator(beta, d, p, q1, q2)

        # Compute distance and update prices
        dist = np.max(np.abs(p - Tkp))
        p[:] = Tkp[:]

    return Tkp
        

In [7]:
phk = hk_compute_price_function(beta, d, Q1, Q2)

In [8]:
print("Price under homogenous beliefs Q1", p1)
print("Price under homogenous beliefs Q2", p2)
print("Price under heterogeneous beliefs", phk)

Price under homogenous beliefs Q1 [ 40.75976974  35.96087782  34.57199775]
Price under homogenous beliefs Q2 [ 23.50112992  21.0988263   20.28394791]
Price under heterogeneous beliefs [ 40.75976974  35.96087782  34.57199775]


This wasn't a very interesting example because the agent with beliefs $Q1$ always values the asset more than the other agent. However, if we change the beliefs a little, in particular if we make agent 2 think that he is very likely to go from a mild recession back to normal growth, then we are able to get a bubble -- A bubble in the sense that both agents value the asset more than they think it is worth.

In [9]:
Q1 = np.array([[0.971, 0.029, 0.000],
               [0.145, 0.778, 0.077],
               [0.000, 0.508, 0.492]])

Q2 = np.array([[0.871, 0.029, 0.100],
               [0.900, 0.009, 0.01],
               [0.000, 0.508, 0.492]])

d = np.array([1.0, 0.0, 0.0])

beta = 0.98

In [10]:
p1 = compute_price_function(beta, d, Q1)
p2 = compute_price_function(beta, d, Q2)
phk = hk_compute_price_function(beta, d, Q1, Q2)

print("Price under homogenous beliefs Q1", p1)
print("Price under homogenous beliefs Q2", p2)
print("Price under heterogeneous beliefs", phk)

Price under homogenous beliefs Q1 [ 40.75976974  35.96087782  34.57199775]
Price under homogenous beliefs Q2 [ 26.5891324   24.78574395  23.82846957]
Price under heterogeneous beliefs [ 42.69136681  39.25179728  37.73581544]


Each agent essentially values the asset according to the price computed under homogenous beliefs. However, the agents know they disagree on its value which affects the value at which you can sell the asset. This disagreement can lead (as shown above) to the asset being priced higher than either agent would value it under their homogenous beliefs.